업종별 수익률 대비 변동성을 시각화 하여, 수익률 대비 변동성이 낮은 안정적인 종목의 인사이트를 제공하는 영역

In [42]:
# 데이터 분석을 위한 패키지
import numpy as np
import pandas as pd

In [43]:
# 시각화 패키지
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [44]:
import FinanceDataReader as fdr

In [45]:
import koreanize_matplotlib

In [46]:
import datetime
from dateutil.relativedelta import relativedelta

In [47]:
from data_loader import load_sector_data

In [48]:
# 모든 컬럼,행을 출력하도록 설정
pd.set_option('display.max_columns', None) # None으로 설정하면 모든 컬럼 출력
pd.set_option('display.max_rows', None) #None으로 설정하면 모든 행 출력

In [39]:
# 회귀선의 의미 : 동일한 평균수익률을 가진 다른 업종/종목에 비해 변동성이 적어 리스크가 낮다는 인사이트를 제공할 수 있음
# plotly는 lmplot을 직접적으로 제공하지 않기때문에, scatter기반 회귀선과, 데이터 플롯을 직접 따로 그려야함



def analyze_sector_return_volatility(market=None, sector=None, month_ago=3):
    # 데이터 로드
    df = load_sector_data(market, month_ago=month_ago)  # 사용자 정의 데이터 로드 함수
    

    # 업종이 입력된 경우 해당 업종 내에서 필터링
    if sector is not None:
        if sector not in df['Sector'].unique():
            raise ValueError(f"'{sector}'는(은) 유효한 섹터가 아닙니다. 사용 가능한 섹터: {df['Sector'].unique()}")
        
        df = df[df['Sector'] == sector]
        
    else :
        # 해당 업종의 평균 수익률과 변동성 계산
        df = df.groupby('Sector').agg({
        'TotalReturn': 'mean',
        'Volatility': 'mean'
    }).reset_index().sort_values(by='TotalReturn', ascending=True)    
    
    if df.empty:
        raise ValueError("업종별 평균 수익률과 변동성 데이터가 없습니다.")
    
    # Plotly로 회귀선 데이터를 계산
    fig = px.scatter(
        df,
        x='TotalReturn',
        y='Volatility',
        trendline='ols'
    )
    
    # 회귀선 데이터를 추출
    
    # 다소 난해한 부분 나중에 다시보기
    # px.get_trendline_results(fig)는 Pandas DataFrame을 반환합니다.

    # 형태:

    # 반환된 DataFrame은 각 회귀선에 대한 통계적 정보를 포함하고 있습니다. 일반적으로 이 DataFrame에는 다음과 같은 열이 포함됩니다:
    #     params: 회귀선의 파라미터(절편과 기울기)와 관련된 정보.
    #     r_value: 결정 계수(R² 값), 회귀선의 적합도를 나타냅니다.
    #     p_value: 각 파라미터에 대한 p-값, 통계적 유의성을 나타냅니다.
    #     stderr: 각 파라미터의 표준 오차.
    
    #  px.get_trendline_results(fig) 의 구조
    #           params      r_value     p_value     stderr
    # 0  [intercept, slope]  0.85        0.001        0.05
    
    # 이전에 생성한 fig 객체에서 회귀선의 결과를 가져옴 회귀선의 모델 결과를 포함하는 객체를 가져옵니다. 이 객체에는 회귀선의 절편(intercept)과 기울기(slope) 등의 정보가 포함되어 있음
    trendline_data = px.get_trendline_results(fig).iloc[0].px_fit_results
    
    # trendline_data.params: 회귀선의 파라미터(절편과 기울기)를 반환합니다. 여기서 intercept는 회귀선의 y절편을 의미하고, slope는 회귀선의 기울기를 나타냅니다.
    intercept, slope = trendline_data.params       

    # 회귀선 아래/위 판별 및 색상 추가
    df['Color'] = df.apply(
        # 현재 행의 변동성이 회귀선에서 예측한 변동성보다 낮은지를 확인, 회귀선의 방정식은 y=mx+b 형태이므로, 여기서 intercept는 y절편, slope는 기울기, row["TotalReturn"]는 x값
        lambda row: 'Below' if row['Volatility'] < (intercept + slope * row['TotalReturn']) \
                            else 'Above',axis=1
                            )
    
    # 회귀선 대비 아래에 있는 종목 수익률 내림차순으로 정렬한 df
    below_regression_line_df = df[df['Color'] == 'Below'].sort_values(by='TotalReturn', ascending=False)

    if sector is not None : 
        title = f'{market} 시장 {sector}업종 {month_ago}개월 수익률 대비 변동성 지수'
        trendline_name = f'회귀선({market} 시장 {sector}업종 {month_ago}개월 수익률 대비 평균 변동성)'
        hover_data = {'Name': True}
    else :
        title = f'{market} 시장 업종별 {month_ago}개월 수익률 대비 변동성 지수'
        trendline_name = f'회귀선({market} 시장 {month_ago}개월 수익률 대비 평균 변동성)'
        hover_data = {'Sector': True} 
        
        
    # 최종 Plotly 그래프 생성
    fig = px.scatter(
        df,
        x='TotalReturn',
        y='Volatility',
        color='Color',                  # 회귀선 기준 색상 설정
        hover_data=hover_data,
        title=title,
        labels={'TotalReturn': '평균 수익률', 'Volatility': '평균 변동성'},
        template="plotly_white"
    )

    # 회귀선 다시 추가
    fig.add_trace(go.Scatter(
        x=df['TotalReturn'],
        y=intercept + slope * df['TotalReturn'],
        mode="lines",
        name=trendline_name,
        line=dict(color='black', dash='dash')
    ))
    
    # 제목 가운데 정렬렬
    fig.update_layout(
        title_x=0.5,
        title_font=dict(size=20, color='black', family='Arial')
        ) 
    
    fig.show()
    
    # 변동성 대비 수익률이 좋은 종목 df 반환
    
    recommended_df = below_regression_line_df[below_regression_line_df['TotalReturn'] > 0]
    display(recommended_df)

In [40]:
analyze_sector_return_volatility('KOSPI','IT서비스')

3개월 간의 data를 불러옵니다.


,Code,Name,Volatility,TotalReturn,AvgReturn,Sector,Volume,Amount,Marcap,Color
267,181710,NHN,2.44,5.63,0.095572,IT서비스,26285,464369380,595972070950,Below


In [9]:
analyze_sector_return_volatility('KOSDAQ')

3개월 간의 data를 불러옵니다.


,Sector,TotalReturn,Volatility,Color
63,항공화물운송과물류,36.505000,4.785000,Below
40,운송인프라,17.980000,3.270000,Below
34,손해보험,13.430000,3.200000,Below
18,도로와철도운송,2.550000,1.300000,Below
36,식품과기본식료품소매,1.148000,3.240000,Below
66,"호텔,레스토랑,레저",0.971111,3.070000,Below
1,가구,0.776000,2.734000,Below
28,상업서비스와공급품,0.580952,2.632857,Below
49,전문소매,0.487500,2.005000,Below
20,디스플레이패널,0.380000,0.690000,Below


In [41]:
analyze_sector_return_volatility('ETF','채권')

3개월 간의 data를 불러옵니다.


,Code,Name,Volatility,TotalReturn,AvgReturn,Sector,Volume,Amount,Marcap,Color
222,461500,HANARO 종합채권(AA-이상)액티브,0.25,2.50,0.042936,채권,46002,5210,1151,Below
25,385540,RISE 종합채권(A-이상)액티브,0.20,2.42,0.042160,채권,57689,6253,13441,Below
54,451540,TIGER 종합채권(AA-이상)액티브,0.21,2.39,0.042173,채권,16333,912,6487,Below
96,454780,히어로즈 종합채권(AA-이상)액티브,0.23,2.32,0.038327,채권,199,21,3557,Below
47,356540,ACE 종합채권(AA-이상)KIS액티브,0.24,2.28,0.040919,채권,4419,471,7466,Below
43,436140,SOL 종합채권(AA-이상)액티브,0.21,2.27,0.039899,채권,37,4,8322,Below
117,451000,PLUS 종합채권(AA-이상)액티브,0.25,2.27,0.042024,채권,20,2,2658,Below
10,273130,KODEX 종합채권(AA-이상)액티브,0.23,2.26,0.041493,채권,62394,7283,28356,Below
438,272570,RISE 중장기국공채액티브,0.14,1.99,0.032178,채권,29,3,298,Below
131,438330,TIGER 투자등급회사채액티브,0.12,1.98,0.032577,채권,7558,869,2302,Below
